In [1]:
import os
import re
import numpy as np
from sklearn.metrics import confusion_matrix

import torch
from torch.nn import MaxPool1d

In [4]:
dir_name = "causal_eval_kinetics_r50_i3d_nonlocal_8x8_fullyConv"
files_dir = "/Volumes/UBUNTU 18_0/PAKOVA/outputs/Kinetics/" + dir_name

result_files = sorted([os.path.join(files_dir, f) for f in os.listdir(files_dir) if f.endswith('.txt')])

# split_text = []
text = ''
for fid, fname in enumerate(result_files):
    print(fname)
    with open(fname, 'r', encoding ="ISO-8859-1") as infile:
        for line in infile:
            if line[0].isalpha():
                line = '| ' + line
            text += line.strip() + ' '

/Volumes/UBUNTU 18_0/PAKOVA/outputs/Kinetics/causal_eval_kinetics_r50_i3d_nonlocal_8x8_fullyConv/causal_eval_kinetics_r50_i3d_nonlocal_8x8_fullyConv_0.txt
/Volumes/UBUNTU 18_0/PAKOVA/outputs/Kinetics/causal_eval_kinetics_r50_i3d_nonlocal_8x8_fullyConv/causal_eval_kinetics_r50_i3d_nonlocal_8x8_fullyConv_1.txt
/Volumes/UBUNTU 18_0/PAKOVA/outputs/Kinetics/causal_eval_kinetics_r50_i3d_nonlocal_8x8_fullyConv/causal_eval_kinetics_r50_i3d_nonlocal_8x8_fullyConv_10.txt
/Volumes/UBUNTU 18_0/PAKOVA/outputs/Kinetics/causal_eval_kinetics_r50_i3d_nonlocal_8x8_fullyConv/causal_eval_kinetics_r50_i3d_nonlocal_8x8_fullyConv_11.txt
/Volumes/UBUNTU 18_0/PAKOVA/outputs/Kinetics/causal_eval_kinetics_r50_i3d_nonlocal_8x8_fullyConv/causal_eval_kinetics_r50_i3d_nonlocal_8x8_fullyConv_12.txt
/Volumes/UBUNTU 18_0/PAKOVA/outputs/Kinetics/causal_eval_kinetics_r50_i3d_nonlocal_8x8_fullyConv/causal_eval_kinetics_r50_i3d_nonlocal_8x8_fullyConv_13.txt
/Volumes/UBUNTU 18_0/PAKOVA/outputs/Kinetics/causal_eval_kinetics_

In [ ]:
# This code is to connect broken lines of new code (400 preds)
split_text = text.split('|')

if split_text[0] == '':
    split_text = split_text[1:]
    
split_text2 = []
for i in range(0, len(split_text), 3):
    res = split_text[i:i+3]
    split_text2.append(res)
        
        
%xdel split_text
split_text = split_text2

In [4]:
def get_full_predictions(ids, preds, num_classes=400):
    miss = (1 - preds.sum()) / (num_classes - preds.shape[0])
    full_preds = np.full((num_classes), miss)
    full_preds[ids] = preds
    return full_preds


def divide_per_clip(split_text):
    clips_targets = []
    clips_labels = []
    clips_prediction = []
    clips_frames = []

    video_name = None
    for i, [video_path, target, pred] in enumerate(split_text):
        video = re.sub('()_\\d{6}', '', video_path.strip())

        if video != video_name:            
            # new video! But first, save old video
            if i > 0:
                video_targets = np.array(video_targets)
                video_labels = np.array(video_labels)
                video_prediction = np.array(video_prediction)
                
                num_frames = video_prediction.shape[0]
                if num_frames < 10:
                    repeat_ids = np.round(np.linspace(0, num_frames - 1, 10)).astype(int)
                    video_targets = video_targets[repeat_ids]
                    video_labels = video_labels[repeat_ids]
                    video_prediction = video_prediction[repeat_ids]
                
                clips_targets.append(video_targets)
                clips_labels.append(video_labels)
                clips_prediction.append(video_prediction)
                clips_frames.append(video_frames)
                    
            # start new one
            video_targets = []
            video_labels = []
            video_prediction = []
            video_frames = []
            video_name = video
            
        video_targets.append(int(target))

        preds = np.fromstring(pred, dtype=float, sep=' ')
        video_prediction.append(preds)
        
        labels = np.argsort(preds)[-1:-6:-1]
        video_labels.append(labels)
        
        video_frames.append(video_path.strip())
        
    # Saving last frame of last video
    video_targets = np.array(video_targets)
    video_labels = np.array(video_labels)
    video_prediction = np.array(video_prediction)
    
    num_frames = video_prediction.shape[0]
    if num_frames < 10:
        repeat_ids = np.round(np.linspace(0, num_frames - 1, 10)).astype(int)
        video_targets = video_targets[repeat_ids]
        video_labels = video_labels[repeat_ids]
        video_prediction = video_prediction[repeat_ids]

    clips_targets.append(video_targets)
    clips_labels.append(video_labels)
    clips_prediction.append(video_prediction)
    clips_frames.append(video_frames)
    
                
    return clips_targets, clips_labels, clips_prediction, clips_frames

def select_n_clips(video_classes, n=10):
    num_frames = len(video_classes)
    ids = np.linspace(0, num_frames-1, n, dtype=int)
    
    return video_classes[ids]


In [5]:
# Dividing per clip
clip_targets, clip_labels, clip_pred, clip_frames = divide_per_clip(split_text)


# Getting all targets and labels
all_targets = np.hstack(clip_targets)
all_labels = np.vstack(clip_labels)


# Getting result of 10 clips
clip_pred_10 = [select_n_clips(np.array(clip_data), n=10) for clip_data in clip_pred]

clip_pred_10_mean = np.array([t_c.mean(axis=0) for t_c in clip_pred_10])
clip_gt_10 = np.array([clip_data[0] for clip_data in clip_targets])


# Dividing all frames in percentages
p_clip_targets = np.array([np.array_split(ct, 10) for ct in clip_targets])
p_clip_labels = np.array([np.array_split(cl, 10) for cl in clip_labels])
p_clip_pred = np.array([np.array_split(cp, 10) for cp in clip_pred])


In [6]:
num_videos = p_clip_targets.shape[0]
num_percent = p_clip_targets.shape[1]
num_topk = 5

percentage_targets = np.zeros((num_videos, num_percent))
percentage_pred = np.zeros((num_videos, num_percent, num_topk))

for clip_id, (p_targets, p_preds) in enumerate(zip(p_clip_targets, p_clip_pred)):
    for p_id in range(10):
        pt = np.hstack(p_targets[:p_id+1])
        pp = np.vstack(p_preds[:p_id+1])
        assert len(set(pt)) == 1  # Checking if all targets match
        percentage_targets[clip_id, p_id] = pt[0]
        preds = torch.tensor(pp)
        mean_pp = preds.mean(0)
        _, topk_labels = torch.topk(mean_pp, 5)
        percentage_pred[clip_id, p_id] = topk_labels.numpy()

In [7]:
def get_top_predictions(video_pred, video_labels):
    # TOP1
    top1_pred = [p[0] for p in video_pred]
    # TOP 5
    top5_pred = []
    for l, p in zip(video_labels, video_pred):
        if l in p:
            top5_pred.append(l)
        else:
            top5_pred.append(p[0])
    return top1_pred, top5_pred


def per_class_accuracy(predictions, labels):
    cf = confusion_matrix(labels, predictions).astype(float)

    cls_cnt = cf.sum(axis=1)
    cls_hit = np.diag(cf)
    return np.nanmean(cls_hit / cls_cnt), cls_hit.sum()/cls_cnt.sum()


def get_per_frame_percent_results(p_targets, p_preds):
    per_frame_percentage_results = []
    for p_id in range(10):
        p_t = p_targets[:, p_id]
        p_p = p_preds[:, p_id]
        top1_pred, top5_pred = get_top_predictions(p_p, p_t)

        pcls_acc1, cls_acc1 = per_class_accuracy(top1_pred, p_t)
        pcls_acc5, cls_acc5 = per_class_accuracy(top5_pred, p_t)
        per_frame_percentage_results.append([cls_acc1, cls_acc5])
        print('{}% - T1: {:.1%} (per-class: {:.1%}) | T5: {:.1%} (per-class: {:.1%})'.format(
            (p_id+1)*10, cls_acc1, pcls_acc1, cls_acc5, pcls_acc5))
    return np.array(per_frame_percentage_results)

In [8]:
# All frames results

top1_pred_all, top5_pred_all = get_top_predictions(all_labels, all_targets)

pcls_acc1_all, cls_acc1_all = per_class_accuracy(top1_pred_all, all_targets)
pcls_acc5_all, cls_acc5_all = per_class_accuracy(top5_pred_all, all_targets)

print('T1: {:.2%} | T5: {:.2%}'.format(cls_acc1_all, cls_acc5_all))
print('T1: {:.2%} | T5: {:.2%} (per class)'.format(pcls_acc1_all, pcls_acc5_all))

T1: 65.13% | T5: 85.06%
T1: 64.57% | T5: 84.66% (per class)


In [9]:
# 10 clips results

clip_top5_10 = []
for mean_10 in clip_pred_10_mean:
    _, topk_labels = torch.topk(torch.tensor(mean_10), 5)
    clip_top5_10.append(topk_labels.numpy())
    
clip_top5_10 = np.array(clip_top5_10)

top1_pred, top5_pred = get_top_predictions(clip_top5_10, clip_gt_10)

pcls_acc1_10, cls_acc1_10 = per_class_accuracy(top1_pred, clip_gt_10)
pcls_acc5_10, cls_acc5_10 = per_class_accuracy(top5_pred, clip_gt_10)

print('T1: {:.2%} | T5: {:.2%}'.format(cls_acc1_10, cls_acc5_10))
print('T1: {:.2%} | T5: {:.2%} (per class)'.format(pcls_acc1_10, pcls_acc5_10))

T1: 71.74% | T5: 90.06%
T1: 71.67% | T5: 90.02% (per class)


In [10]:
# All frames mean percentage results

per_frame_percent_res = get_per_frame_percent_results(percentage_targets, percentage_pred)

10% - T1: 64.3% (per-class: 64.2%) | T5: 84.7% (per-class: 84.7%)
20% - T1: 65.8% (per-class: 65.7%) | T5: 86.0% (per-class: 85.9%)
30% - T1: 67.3% (per-class: 67.2%) | T5: 87.3% (per-class: 87.2%)
40% - T1: 68.8% (per-class: 68.8%) | T5: 88.2% (per-class: 88.1%)
50% - T1: 69.7% (per-class: 69.6%) | T5: 89.0% (per-class: 88.9%)
60% - T1: 70.3% (per-class: 70.2%) | T5: 89.4% (per-class: 89.4%)
70% - T1: 70.9% (per-class: 70.8%) | T5: 89.8% (per-class: 89.8%)
80% - T1: 71.4% (per-class: 71.3%) | T5: 90.0% (per-class: 89.9%)
90% - T1: 71.6% (per-class: 71.6%) | T5: 90.1% (per-class: 90.1%)
100% - T1: 71.9% (per-class: 71.9%) | T5: 90.2% (per-class: 90.1%)


In [11]:
results = np.concatenate((np.array([[cls_acc1_all, cls_acc5_all],[cls_acc1_10, cls_acc5_10]]), per_frame_percent_res))

In [12]:
output_file = files_dir + '_results'
np.save(output_file, results, allow_pickle=False)

In [15]:
results_baseline8_centerCrop = np.load(output_file+'.npy')

In [32]:
results_baseline32_centerCrop

NameError: name 'results_baseline32_centerCrop' is not defined

In [30]:
results_nl32_centerCrop

NameError: name 'results_nl32_centerCrop' is not defined

In [16]:
results_baseline8_centerCrop

array([[0.65134888, 0.85057376],
       [0.71735596, 0.90061986],
       [0.64277004, 0.8471192 ],
       [0.65750432, 0.85951631],
       [0.67254344, 0.87262473],
       [0.6884971 , 0.88166853],
       [0.69657555, 0.88969617],
       [0.70297734, 0.8944213 ],
       [0.70912509, 0.89848593],
       [0.7137486 , 0.89980693],
       [0.71649223, 0.90117874],
       [0.71923585, 0.90173763]])

In [14]:
results_baseline8_fullyConv

array([[0.67416182, 0.8663481 ],
       [0.73203943, 0.90747891],
       [0.66421095, 0.85931308],
       [0.67686211, 0.87257393],
       [0.6923585 , 0.88354842],
       [0.70414592, 0.89304949],
       [0.71166548, 0.89807946],
       [0.71837212, 0.9015344 ],
       [0.72482471, 0.90458287],
       [0.72919419, 0.90717407],
       [0.73153135, 0.90829184],
       [0.73341124, 0.90940961]])

In [37]:
results_nl8_centerCrop

array([[0.66879506, 0.86181267],
       [0.73468143, 0.90956204],
       [0.66156895, 0.85850015],
       [0.67737019, 0.871761  ],
       [0.68936084, 0.88425973],
       [0.70252007, 0.89284626],
       [0.71283406, 0.89873997],
       [0.71923585, 0.90356671],
       [0.72431663, 0.9080378 ],
       [0.72975307, 0.9093588 ],
       [0.73290316, 0.91067981],
       [0.73534194, 0.91159435]])

In [14]:
results_nl8_fullyConv

array([[0.69105737, 0.87622711],
       [0.74524947, 0.91504928],
       [0.67955492, 0.86835687],
       [0.69246012, 0.88171934],
       [0.70368865, 0.89330353],
       [0.71659384, 0.90112793],
       [0.7234529 , 0.90585306],
       [0.730566  , 0.90991769],
       [0.7357484 , 0.91245808],
       [0.74011787, 0.91535413],
       [0.74331877, 0.91631948],
       [0.74438573, 0.91723402]])